# 原子操作
前两节中我们学习了互斥锁,读写锁以及基于它们的条件变量。

互斥锁是一个很有用的同步工具，它可以保证每一时刻进入临界区的goroutine只有一个。读写锁对共享资源的写操作和读操作则区别看待，并消除了读操作之间的互斥。

通过对互斥锁的合理使用，我们可以使一个goroutine在执行临界区中的代码时，不被其他的goroutine打扰。不过，虽然不会被打扰，但是它仍然可能会被中断(interruption)

## 前导内容：原子性执行与原子操作

我们已经知道，对于一个Go程序来说，Go语言运行时系统中的调度器会恰当地安排其中所有的goroutine的运行。不过，在同一时刻，只可能有少数的goroutine真正地处于运行状态，并且这个数量只会与M的数量一致，而不会随着G的增多而增长。

所以，为了公平起见，调度器总是会频繁地换上或换下这些goroutine。换上的意思是，让一个goroutine由非运行状态转为运行状态，并促使其中的代码在某个CPU核心上执行。

换下的意思正好相反，即：使一个goroutine中的代码中断执行，并让它由运行状态转为非运行状态。

这个中断的时机有很多，任何两条语句执行的间隙，甚至在某条语句执行的过程中都是可以的。

即使这些语句在临界区之内也是如此。所以，我们说，互斥锁虽然可以保证临界区中代码的串行执行，但却不能保证这些代码执行的**原子性（atomicity）**。

在众多的同步工具中，真正能够保证**原子性执行的只有原子操作（atomic operation）**。原子操作在进行的过程中是不允许**中断**的。在底层，这会由CPU提供**芯片级别**的支持，所以绝对有效。即使在拥有多CPU核心，或者多CPU的计算机系统中，原子操作的保证也是不可撼动的。

这使得原子操作可以完全地消除**竞态条件，并能够绝对地保证并发安全性**。并且，它的执行速度要比其他的同步工具快得多，通常会高出好几个数量级。不过，它的缺点也很明显。

**更具体地说，正是因为原子操作不能被中断，所以它需要足够简单，并且要求快速。**

你可以想象一下，如果原子操作迟迟不能完成，而它又不会被中断，那么将会给计算机执行指令的效率带来多么大的影响。因此，操作系统层面只对针对二进制位或整数的原子操作提供了支持。

Go语言的原子操作当然是基于CPU和操作系统的，所以它也只针对少数数据类型的值提供了原子操作函数。这些函数都存在于标准库代码包`sync/atomic`中。

我一般会通过下面这道题初探一下应聘者对`sync/atomic`包的熟悉程度。

**我们今天的问题是：`sync/atomic`包中提供了几种原子操作？可操作的数据类型又有哪些？**

**这里的典型回答是：**

`sync/atomic`包中的函数可以做的原子操作有：加法（add）、比较并交换（compare and swap，简称CAS）、加载（load）、存储（store）和交换（swap）。

这些函数针对的数据类型并不多。但是，对这些类型中的每一个，`sync/atomic`包都会有一套函数给予支持。这些数据类型有：`int32`、`int64`、`uint32`、`uint64`、`uintptr`，以及`unsafe`包中的`Pointer`。不过，针对`unsafe.Pointer`类型，该包并未提供进行原子加法操作的函数。

此外，`sync/atomic`包还提供了一个名为`Value`的类型，它可以被用来存储任意类型的值。

## 问题解析

这个问题很简单，因为答案是明摆在代码包文档里的。不过如果你连文档都没看过，那也可能回答不上来，至少是无法做出全面的回答。

我一般会通过此问题再衍生出来几道题。下面我就来逐个说明一下。

**第一个衍生问题** ：我们都知道，传入这些原子操作函数的第一个参数值对应的都应该是那个被操作的值。比如，`atomic.AddInt32`函数的第一个参数，对应的一定是那个要被增大的整数。可是，这个参数的类型为什么不是`int32`而是`*int32`呢？

回答是：因为原子操作函数需要的是被操作值的指针，而不是这个值本身；被传入函数的参数值都会被复制，像这种基本类型的值一旦被传入函数，就已经与函数外的那个值毫无关系了。这里可以类比值方法和引用方法。

所以，传入值本身没有任何意义。`unsafe.Pointer`类型虽然是指针类型，但是那些原子操作函数要操作的是这个指针值，而不是它指向的那个值，所以需要的仍然是指向这个指针值的指针。

只要原子操作函数拿到了被操作值的指针，就可以定位到存储该值的内存地址。只有这样，它们才能够通过底层的指令，准确地操作这个内存地址上的数据。

**第二个衍生问题**： 用于原子加法操作的函数可以做原子减法吗？比如，`atomic.AddInt32`函数可以用于减小那个被操作的整数值吗？

回答是：当然是可以的。`atomic.AddInt32`函数的第二个参数代表差量，它的类型是`int32`，是有符号的。如果我们想做原子减法，那么把这个差量设置为负整数就可以了。

对于`atomic.AddInt64`函数来说也是类似的。不过，要想用`atomic.AddUint32`和`atomic.AddUint64`函数做原子减法，就不能这么直接了，因为它们的第二个参数的类型分别是`uint32`和`uint64`，都是无符号的，不过，这也是可以做到的，就是稍微麻烦一些。

例如，如果想对`uint32`类型的被操作值18做原子减法，比如说差量是-3，那么我们可以先把这个差量转换为有符号的`int32`类型的值，然后再把该值的类型转换为`uint32`，用表达式来描述就是`uint32(int32(-3))`。

不过要注意，直接这样写会使Go语言的编译器报错，它会告诉你：“常量`-3`不在uint32类型可表示的范围内”，换句话说，这样做会让表达式的结果值溢出。

不过，如果我们先把int32(-3)的结果值赋给变量delta，再把delta的值转换为uint32类型的值，就可以绕过编译器的检查并得到正确的结果了。

最后，我们把这个结果作为`atomic.AddUint32`函数的第二个参数值，就可以达到对`uint32`类型的值做原子减法的目的了。

还有一种更加直接的方式。我们可以依据下面这个表达式来给定`atomic.AddUint32`函数的第二个参数值：

^uint32(-N-1))

其中的`N`代表由负整数表示的差量。也就是说，我们先要把差量的绝对值减去`1`，然后再把得到的这个无类型的整数常量，转换为`uint32`类型的值，最后，在这个值之上做按位异或操作，就可以获得最终的参数值了。

这么做的原理也并不复杂。简单来说，此表达式的结果值的补码，与使用前一种方法得到的值的补码相同，所以这两种方式是等价的。我们都知道，整数在计算机中是以补码的形式存在的，所以在这里，结果值的补码相同就意味着表达式的等价。

## 总结
今天，我们一起学习了`sync/atomic`代码包中提供的原子操作函数和原子值类型。原子操作函数使用起来都非常简单，但也有一些细节需要我们注意。我在主问题的衍生问题中对它们进行了逐一说明。

在下一篇文章中，我们会继续分享原子操作的衍生内容。如果你对原子操作有什么样的问题，都可以给我留言，我们一起讨论，感谢你的收听，我们下期再见。

# 原子操作(下)

**第三个衍生问题： 比较并交换操作与交换操作相比有什么不同？优势在哪里？**

回答是：比较并交换操作即CAS操作，是有条件的交换操作，只有在条件满足的情况下才会进行值的交换。

所谓的交换指的是，把新值赋给变量，并返回变量的旧值。

在进行CAS操作的时候，函数会先判断被操作变量的当前值，是否与我们预期的旧值(期望值)相等。如果相等，它就把新值赋给该变量，并返回`true`以表明交换操作已进行；否则就忽略交换操作，并返回`false`。

可以看到，CAS操作并不是单一的操作，而是一种操作组合。这与其他的原子操作都不同。正因为如此，它的用途要更广泛一些。例如，我们将它与`for`语句联用就可以实现一种简易的自旋锁（spinlock）。

In [1]:
for {
 if atomic.CompareAndSwapInt32(&num2, 10, 0) { //注意这里的10是期望值
 // 如果是 ( num2 == 10 )：它立刻将 num2 的值交换为“新值” (0)，并返回 true。
  fmt.Println("The second number has gone to zero.")
  break
 }
 time.Sleep(time.Millisecond * 500)
}

Cell[1]: Line 1 
 /tmp/gonb_639cbb31/main.go:3:1: expected declaration, found 'for'
 
package main

 for {
 if atomic.CompareAndSwapInt32(&num2, 10, 0) { //注意这里的10是期望值
 // 如果是 ( num2 == 10 )：它立刻将 num2 的值交换为“新值” (0)，并返回 true。

ERROR: parsing go files in TempDir "/tmp/gonb_639cbb31": /tmp/gonb_639cbb31/main.go:3:1: expected declaration, found 'for'

在for语句中的CAS操作可以不停地检查某个需要满足的条件，一旦条件满足就退出for循环。这就相当于，只要条件未被满足，当前的流程就会被一直“阻塞”在这里。

这在效果上与互斥锁有些类似。不过，它们的适用场景是不同的。我们在使用互斥锁的时候，总是假设共享资源的状态会被其他的goroutine频繁地改变。

而for语句加CAS操作的假设往往是：共享资源状态的改变并不频繁，或者，它的状态总会变成期望的那样。这是一种**更加乐观，或者说更加宽松**的做法。

**第四个衍生问题：假设我已经保证了对一个变量的写操作都是原子操作，比如：加或减、存储、交换等等，那我对它进行读操作的时候，还有必要使用原子操作吗？**

回答是：很有必要。其中的道理你可以对照一下读写锁。为什么在读写锁保护下的写操作和读操作之间是互斥的？这是为了防止读操作读到没有被修改完的值，对吗？

如果写操作还没有进行完，读操作就来读了，那么就只能读到仅修改了一部分的值。这显然破坏了值的完整性，读出来的值也是完全错误的。

所以，一旦你决定了要对一个共享资源进行保护，那就要做到完全的保护。不完全的保护基本上与不保护没有什么区别。

好了，上面的主问题以及相关的衍生问题涉及了原子操作函数的用法、原理、对比和一些最佳实践，希望你已经理解了。

由于这里的原子操作函数只支持非常有限的数据类型，所以在很多应用场景下，互斥锁往往是更加适合的。

不过，一旦我们确定了在某个场景下可以使用原子操作函数，比如：只涉及并发地读写单一的整数类型值，或者多个互不相关的整数类型值，那就不要再考虑互斥锁了。

这主要是因为原子操作函数的执行速度要比互斥锁快得多。而且，它们使用起来更加简单，不会涉及临界区的选择，以及死锁等问题。当然了，在使用CAS操作的时候，我们还是要多加注意的，因为它可以被用来模仿锁，并有可能“阻塞”流程。

## 知识扩展

问题：怎样用好`sync/atomic.Value`？

为了扩大原子操作的适用范围，Go语言在1.4版本发布的时候向`sync/atomic`包中添加了一个新的类型`Value`。此类型的值相当于一个容器，可以被用来“原子地”存储和加载任意的值。

`atomic.Value`类型是开箱即用的，我们声明一个该类型的变量（以下简称原子变量）之后就可以直接使用了。这个类型使用起来很简单，它只有两个指针方法：`Store`和`Load`。不过，虽然简单，但还是有一些值得注意的地方的。

首先一点，一旦`atomic.Value`类型的值（以下简称原子值）被真正使用，它就不应该再被复制了。什么叫做“真正使用”呢？

**我们只要用它来存储值了，就相当于开始真正使用了**。`atomic.Value`类型属于结构体类型，而结构体类型属于值类型。

所以，复制该类型的值会产生一个完全分离的新值。这个新值相当于被复制的那个值的一个快照。之后，不论后者存储的值怎样改变，都不会影响到前者，反之亦然。

另外，关于用原子值来存储值，有两条强制性的使用规则。第一条规则，**不能用原子值存储nil**。

也就是说，我们不能把nil作为参数值传入原子值的Store方法，否则就会引发一个panic。

这里要注意，如果有一个接口类型的变量，它的动态值是nil，但动态类型却不是nil，那么它的值就不等于nil。我在前面讲接口的时候和你说明过这个问题。正因为如此，这样一个变量的值是可以被存入原子值的。

第二条规则，**我们向原子值存储的第一个值，决定了它今后能且只能存储哪一个类型的值。**

例如，我第一次向一个原子值存储了一个string类型的值，那我在后面就只能用该原子值来存储字符串了。如果我又想用它存储结构体，那么在调用它的Store方法的时候就会引发一个panic。这个panic会告诉我，这次存储的值的类型与之前的不一致。

你可能会想：我先存储一个接口类型的值，然后再存储这个接口的某个实现类型的值，这样是不是可以呢？

很可惜，这样是不可以的，同样会引发一个panic。**因为原子值内部是依据被存储值的实际类型来做判断的。所以，即使是实现了同一个接口的不同类型，它们的值也不能被先后存储到同一个原子值中。**

遗憾的是，我们无法通过某个方法获知一个原子值是否已经被真正使用，并且，也没有办法通过常规的途径得到一个原子值可以存储值的实际类型。这使得我们误用原子值的可能性大大增加，尤其是在多个地方使用同一个原子值的时候。

下面，我给你几条具体的使用建议。

1. 不要把内部使用的原子值暴露给外界。比如，声明一个全局的原子变量并不是一个正确的做法。这个变量的访问权限最起码也应该是包级私有的。
2. 如果不得不让包外，或模块外的代码使用你的原子值，那么可以声明一个包级私有的原子变量，然后再通过一个或多个公开的函数，让外界间接地使用到它。注意，这种情况下不要把原子值传递到外界，不论是传递原子值本身还是它的指针值。
3. 如果通过某个函数可以向内部的原子值存储值的话，那么就应该在这个函数中先**判断被存储值类型的合法性。若不合法，则应该直接返回对应的错误值，从而避免panic的发生**。
4. 如果可能的话，我们可以把原子值封装到一个数据类型中，比如一个结构体类型。这样，我们既可以通过该类型的方法更加安全地存储值，又可以在该类型中包含可存储值的合法类型信息。
除了上述使用建议之外，我还要再特别强调一点：**尽量不要向原子值中存储引用类型的值。因为这很容易造成安全漏洞。**请看下面的代码：

In [ ]:
var box6 atomic.Value
v6 := []int{1, 2, 3}
box6.Store(v6)
v6[1] = 4 // 注意，此处的操作不是并发安全的！

我把一个`[]int`类型的切片值`v6`,存入了原子值`box6`。注意，切片类型属于引用类型。所以，我在外面改动这个切片值，就等于修改了`box6`中存储的那个值。这相当于绕过了原子值而进行了非并发安全的操作。那么，应该怎样修补这个漏洞呢？可以这样做：

In [ ]:
store := func(v []int) {
 replica := make([]int, len(v))
 copy(replica, v)
 box6.Store(replica)
}
store(v6)
v6[2] = 5 // 此处的操作是安全的。

我先为切片值`v6`创建了一个完全的副本。这个副本涉及的数据已经与原值毫不相干了。然后，我再把这个副本存入`box6`。如此一来，无论我再对`v6`的值做怎样的修改，都不会破坏`box6`提供的安全保护。

以上，就是我要告诉你的关于`atomic.Value`的注意事项和使用建议。你可以在demo64.go文件中看到相应的示例。

## 总结

我们把这两篇文章一起总结一下。相对于原子操作函数，原子值类型的优势很明显，但它的使用规则也更多一些。首先，在首次真正使用后，原子值就不应该再被复制了。

其次，原子值的`Store`方法对其参数值（也就是被存储值）有两个强制的约束。一个约束是，参数值不能为`nil`。另一个约束是，参数值的类型不能与首个被存储值的类型不同。也就是说，一旦一个原子值存储了某个类型的值，那它以后就只能存储这个类型的值了。

基于上面这几个注意事项，我提出了几条使用建议，包括：不要对外暴露原子变量、不要传递原子值及其指针值、尽量不要在原子值中存储引用类型的值，等等。与之相关的一些解决方案我也一并提出了。希望你能够受用。

原子操作明显比互斥锁要更加轻便，但是限制也同样明显。所以，我们在进行二选一的时候通常不会太困难。但是原子值与互斥锁之间的选择有时候就需要仔细的考量了。不过，如果你能牢记我今天讲的这些内容的话，应该会有很大的助力。

## 思考题：
今天的思考题只有一个，那就是：如果要对原子值和互斥锁进行二选一，你认为最重要的三个决策条件应该是什么？

- 被保护的数据是什么类型的？是值类型的还是引用类型的？
- 操作被保护数据的方式是怎样的？是简单的读和写还是更复杂的操作？
- 操作被保护数据的代码是集中的还是分散的？如果是分散的，是否可以变为集中的？
在搞清楚上述问题（以及你关注的其他问题）之后，优先使用原子值。